In [1]:
import time

# pour un ordi avec GPU :

import GPUtil
import torch
print(torch.cuda.is_available())
!set CUDA_PATH="/usr/local/cuda-12"

import pandas as pd
import spacy

GPUtil.showUtilization()

True
| ID | GPU | MEM |
------------------
|  0 | 17% |  2% |


In [2]:
df = pd.read_csv("df_CC_cleaned.csv", sep = ",", encoding = "utf-8", dtype= str)

In [3]:
# df = df.sample(100)

In [4]:
#  j'enlève les colonnes que je n'utiliserai pas, pour réduire le poids du fichier 
# df = df.drop(["Contribution", "lemmas_only_VERB_ADJ_ADV_NOUN", "postags_only_VERB_ADJ_ADV_NOUN", "lemmas_only_VERB_ADJ_ADV_NOUN_without_stopwords"], axis=1)
df = df.drop(["Contribution"], axis=1)


In [5]:
# vérifier l'état des données

if df.isnull().values.any() :
    print("y a t il des nan? ->",df.isnull().values.any())
    print("combien y a t il de nan? ->",df.isnull().values.sum())
    print("où sont les null? ->\n",df.isnull().sum())
    print(df[df.isnull().T.any()])
else :
    print("il n'y a pas de NaN")

print("quels types pour chaque colonne?")    
for column in df.columns:
    print(pd.api.types.infer_dtype(df[column]))
print("y a-t-il des types mixés dans les données?")
print(df[column][df[column].apply(lambda x: isinstance(x, type))])
print(df._is_mixed_type)
print(df.dtypes.nunique()>1)

il n'y a pas de NaN
quels types pour chaque colonne?
string
string
string
string
string
string
string
string
string
string
y a-t-il des types mixés dans les données?
Series([], Name: clean, dtype: object)
False
False


In [18]:
# si on souhaite print des contributions dans toute leur longueur (ne pas appliquer sur tout le corpus)

def print_full(df):
    pd.set_option('display.max_colwidth', 3000)
    # pd.set_option('display.max_rows', None)
    # pd.set_option('display.max_columns', None)
    # pd.set_option('display.width', 2000)
    # pd.set_option('display.float_format', '{:20,.2f}'.format)
    # pd.set_option('display.max_colwidth', None)
    print(df)
    # pd.reset_option('display.max_rows')
    # pd.reset_option('display.max_columns')
    # pd.reset_option('display.width')
    # pd.reset_option('display.float_format')
    # pd.reset_option('display.max_colwidth')
    pd.reset_option('display.max_colwidth')

# autres options :


In [6]:
# rename the columns without spaces (header names, not my best idea)

df = df.rename(columns={"Date de réception" : "Date_de_reception", "Code postal" : "Code_postal", "Code INSEE" : "Code_INSEE",
           "Numéro d'ordre arbitraire" : "Numero_d_ordre_arbitraire", "Type Graphie TT" : "Type_Graphie_TT" , "Numéro de page" : "Numero_de_page",
           "Numéro séquentiel" : "Numero_sequentiel"})

In [9]:
nlp = spacy.load("fr_core_news_md", disable = ["ner"])
    # nlp.max_length = 3000000 #this is possible as we're not using parser or ner.



In [12]:
# config = en commentaire, l'ensemble de règles sentencize(), sinon le modèle sents

def sentencize(document, subset, column_name):
    if subset :
        document = document.sample(subset)

    t0 = time.time()
    all_sentences_df = pd.DataFrame()

    for doc, infos_doc in zip(nlp.pipe(document[column_name], batch_size=128), document.itertuples()):
        sentence_list = [sent.text.strip() for sent in doc.sents]

        sentence_df = pd.DataFrame({"phrases" : sentence_list})

        metadata = pd.DataFrame([infos_doc]*len(sentence_list))

        sentence_df = pd.concat([sentence_df,metadata], axis=1)

        all_sentences_df = pd.concat([all_sentences_df,sentence_df])

        # deleting the lists for clearing, to force python garbage collector to *actually* collect

        del sentence_df
        del sentence_list
        del metadata

            
    t3 = time.time()
    print("Total time: {}".format(t3-t0))
    #  ? all_sentences_df = all_sentences_df.drop(labels = "Contribution", axis=1)
    return(all_sentences_df)


In [24]:
df_test = df[0:2999]

In [25]:
# DONOT FORGET TO CHOOSE A SUBSET IF TESTING

sentences = sentencize(df_test[["Catégorie","Date_de_reception","Code_postal","Code_INSEE","Numero_d_ordre_arbitraire",
                          "Type_Graphie_TT","Numero_de_page","Numero_sequentiel","clean"]],subset = None, column_name = "clean")

Total time: 20.226872444152832


In [26]:
# with GPU, Total time: 3189.847151994705 (without if and with fr_core_news_md)
# avec stopwords

# test avec l'ensemble de règles sentencize() :
# sentences = sentencize(df,12, column_name = "clean")

In [27]:
# adding a column to the df
# renumber the index column (not accurate bc origin from each concatenation)

sentence_df = sentences.reset_index()

# drop the secoond index column

sentence_df = sentence_df.drop(labels = "index", axis=1)

In [30]:
print(sentence_df.shape)

(45960, 11)
0        -supprimer la retraite du président de la répu...
1        -supprimer la retraite du président de la répu...
2        -supprimer la retraite du président de la répu...
3        -supprimer la retraite du président de la répu...
4        -supprimer la retraite du président de la répu...
                               ...                        
45955    lecture d'un message-mail orange https  webmai...
45956    lecture d'un message-mail orange https  webmai...
45957    lecture d'un message-mail orange https  webmai...
45958    lecture d'un message-mail orange https  webmai...
45959    lecture d'un message-mail orange https  webmai...
Name: clean, Length: 45960, dtype: object


In [18]:
sentence_df.to_csv("sentences_df.csv", index = False)